# Practica 1: Predicción de la producción de energía eólica con SCIKIT-LEARN

## Análisis exploratorio de datos (EDA)

Cargamos el dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
wind_ava = pd.read_csv('dataset/wind_ava.csv.gz', compression="gzip")

Seleccionamos la columna "datetime" como indice tras convertirla a un objeto DateTime y descartamos todas las columnas que no sean "energy" (nuestra columna de predicción) y las variables de la localización 13 Sotavento.

In [3]:
wind_ava["datetime"] = pd.to_datetime(wind_ava["datetime"])
wind_ava.set_index("datetime", inplace=True)
for c in wind_ava.columns:
  if not c.endswith(".13") and c != "energy":
    wind_ava.drop(c, axis=1, inplace=True)


A continuación, mostramos los datos estadísticos de las variables restantes. Se puede observar que existen 4748 instancias y 22 características. Todas las variables son numéricas. Además podemos observar que todas las desviaciones típicas (std) son distintas de 0, por lo que ninguna columna es constante. El objetivo es predecir la cantidad de energía generada por lo que este es un problema de regresión.

In [4]:
wind_ava.describe()

,energy,p54.162.13,p55.162.13,cape.13,p59.162.13,lai_lv.13,lai_hv.13,u10n.13,v10n.13,sp.13,...,t2m.13,stl2.13,stl3.13,iews.13,inss.13,stl4.13,fsr.13,flsr.13,u100.13,v100.13
count,4748.000000,4.748000e+03,4748.000000,4748.000000,4.748000e+03,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,...,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000
mean,693.126247,2.489477e+06,16.008810,31.166541,1.706692e+06,2.815222,2.576284,0.386215,0.120528,97820.301287,...,285.689253,286.663838,286.665988,0.074229,0.049971,286.668152,0.413677,-5.908467,0.447175,0.328204
std,665.531609,4.482599e+04,6.552216,121.758977,1.466953e+06,0.397377,0.116434,3.100583,3.016766,713.689654,...,6.163483,5.547947,4.582827,0.367013,0.379014,3.552873,0.007602,0.094359,4.841730,4.667552
min,0.010000,2.358748e+06,1.650268,0.000000,5.610341e+04,2.323973,2.425866,-8.619823,-8.867441,93770.364813,...,268.970603,275.461648,278.389271,-1.714897,-1.438829,280.875389,0.364805,-6.130465,-11.879053,-13.043453
25%,144.170000,2.458543e+06,11.203264,0.000000,6.563209e+05,2.425944,2.461630,-1.950008,-2.050920,97459.369264,...,281.458939,282.287394,282.689506,-0.126880,-0.148495,283.405549,0.410027,-5.977599,-3.836853,-3.256194
50%,465.305000,2.490478e+06,15.543441,1.004148,1.239176e+06,2.758857,2.560520,0.048820,-0.191853,97861.147677,...,285.395453,286.191880,286.204914,0.010551,-0.003569,286.591659,0.410917,-5.947220,0.282399,-0.389416
75%,1089.375000,2.525134e+06,20.214077,14.143328,2.296548e+06,3.205385,2.688526,2.641779,2.016289,98251.478418,...,289.740438,291.345311,290.989045,0.232378,0.172325,290.024705,0.417070,-5.858848,4.187953,3.614395
max,2792.550000,2.580387e+06,39.230807,2311.662152,1.110694e+07,3.450745,2.762992,12.974802,11.699814,99917.733093,...,305.000640,299.556292,295.639998,2.842552,2.366522,292.808658,0.428914,-5.618172,18.964137,16.913033


Calculamos los missing values, tanto NULL como NaN, de cada columna y no existe ninguno de estos datos en todo el dataset. 

In [5]:
wind_ava.isna().sum()

energy        0
p54.162.13    0
p55.162.13    0
cape.13       0
p59.162.13    0
lai_lv.13     0
lai_hv.13     0
u10n.13       0
v10n.13       0
sp.13         0
stl1.13       0
u10.13        0
v10.13        0
t2m.13        0
stl2.13       0
stl3.13       0
iews.13       0
inss.13       0
stl4.13       0
fsr.13        0
flsr.13       0
u100.13       0
v100.13       0
dtype: int64

In [6]:
wind_ava.isnull().sum()

energy        0
p54.162.13    0
p55.162.13    0
cape.13       0
p59.162.13    0
lai_lv.13     0
lai_hv.13     0
u10n.13       0
v10n.13       0
sp.13         0
stl1.13       0
u10.13        0
v10.13        0
t2m.13        0
stl2.13       0
stl3.13       0
iews.13       0
inss.13       0
stl4.13       0
fsr.13        0
flsr.13       0
u100.13       0
v100.13       0
dtype: int64

## Selección de Modelo y Optimización de Hiperparámetros

### Metodología de evaluación

Nuestro dataset es una serie temporal por lo que la validación y medición del rendimiento debe tener esto en cuenta. Tenemos datos de 9 años, y nuestros splits serán anuales. Nuestra outer validation, usará los dos últimos años como test set y la inner usaremos TimeSeriesSplits para hacer el ajuste de hiperparámetros.
Para la evaluación de los modelos usaremos la raíz de error cuadrático medio (RMSE), que lo haremos a traves de la función que ofrece scikit-learn llamada root_mean_squared_error.

##### Los datos no son 9 años y no son continuos


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error, r2_score

In [8]:
# Outer Loop
test_size = len([0 for x in wind_ava.index if x.year >=2009])
train, test = train_test_split(wind_ava, test_size=test_size, shuffle=False)
X_train = train.drop(['energy'], axis='columns')
y_train = train[['energy']]
X_test = test.drop(['energy'], axis='columns')
y_test = test[['energy']]
#Inner Loop
tss = TimeSeriesSplit(n_splits=3)

In [9]:
for k, data in enumerate(tss.split(X_train, y_train)):
  cv_train, cv_val = data
  print(f"En la iteración {k+1}, los índices son:")
  print(f"  Train: {cv_train[0]} - {cv_train[-1]}")
  print(f"  Val: {cv_val[0]} - {cv_val[-1]}")

En la iteración 1, los índices son:
  Train: 0 - 958
  Val: 959 - 1914
En la iteración 2, los índices son:
  Train: 0 - 1914
  Val: 1915 - 2870
En la iteración 3, los índices son:
  Train: 0 - 2870
  Val: 2871 - 3826


### Selección de Scaler

En esta sección elegiremos el mejor scaler usando KNN haciendo validación cruzada con nuestro conjunto de entrenamiento y una pequeña optimización de hiperparámetros (HPO). 

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV

scaler_scores = {}
param_grid = {"knn__n_neighbors":[3,5,7,9,11,13,15]}
param_grid_b = {"n_neighbors":[3,5,7,9,11,13,15]}

In [11]:
# Standard Scaler
pipeline_std = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])
grid_std = GridSearchCV(pipeline_std, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_std.fit(X_train, y_train)
scaler_scores["StandardScaler"] = -grid_std.best_score_

In [12]:
# MinMax Scaler
pipeline_minmax = Pipeline([
    ("scaler", MinMaxScaler()),
    ("knn", KNeighborsRegressor())
])
grid_minmax = GridSearchCV(pipeline_minmax, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_minmax.fit(X_train, y_train)
scaler_scores["MinMaxScaler"] = -grid_minmax.best_score_

In [13]:
# Robust Scaler
pipeline_robust = Pipeline([
    ("scaler", RobustScaler()),
    ("knn", KNeighborsRegressor())
])
grid_robust = GridSearchCV(pipeline_robust, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_robust.fit(X_train, y_train)
scaler_scores["RobustScaler"] = -grid_robust.best_score_

In [14]:
# No scaler
grid_knn = GridSearchCV(KNeighborsRegressor(), param_grid_b, cv=tss, scoring="neg_root_mean_squared_error")
grid_knn.fit(X_train, y_train)
scaler_scores["KNN w/o scaler"] = -grid_knn.best_score_

In [15]:
for name, score in scaler_scores.items():
    print(f"{name}: {score}")

StandardScaler: 434.8935360208871
MinMaxScaler: 472.164886672966
RobustScaler: 452.59145275866126
KNN w/o scaler: 609.373666055656


En estos resultados podemos observar en primer lugar, la diferencia notable que produce el escalado de datos en modelos de KNN. Además para nuestro dataset el método de escalado que mejor rendimiento ofrece es el Standard, que realiza la conocida normalización de la distribución normal que consiste en restar la media y dividir por la varianza del conjunto de entrenamiento. Este escalador es el que usaremos de aqui en adelante para los modelos que se beneficien de ello como KNN o SVM.

### Regresor dummy

Explicacion dummy

In [16]:
from sklearn.dummy import DummyRegressor
dummy = DummyRegressor().fit( X_train, y_train)
prediction = dummy.predict(X_test)
print("RMSE:", root_mean_squared_error(y_test, prediction))
print("R2:", r2_score(y_test, prediction))

RMSE: 666.7611633784188
R2: -3.880265958544626e-06


### Evaluación de KNN

En esta sección, evaluaremos un modelo basado en KNN y ajustaremos los hiperparámetros más importantes, midiendo los tiempos de entrenamiento.

Inicialmente, evaluaremos el modelo con los parámetros establecidos por defecto. Esto es:
    - n_neighbors = 5
    - weights='uniform'
    - algorithm='auto'
    - leaf_size=30
    - p=2
    - metric='minkowski'
    - metric_params=None
    - n_jobs=None

In [17]:
import time
data = [[]] # Usado para almacenar los resultados
initial_time = time.time()

pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('knn', KNeighborsRegressor())]
)

pipe.fit(X_train, y_train)
prediction = pipe.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data[-1].append("KNN")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 434.509641696542 R2: 0.5753218376408425 Time: 0.12599468231201172


explicacion resultados y grid search

In [18]:
initial_time = time.time()
param_grid = {"knn__n_neighbors":list(range(5,17,2)), "knn__weights":["uniform", "distance"], "knn__p":[1,2]}
pipeline_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])
grid_knn = GridSearchCV(pipeline_knn, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_knn.fit(X_train, y_train)
prediction = grid_knn.best_estimator_.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_knn.best_params_)

RMSE: 414.2416225177235 R2: 0.6140166579792947 Time: 3.288888931274414 Params: {'knn__n_neighbors': 11, 'knn__p': 1, 'knn__weights': 'distance'}


### Evaluación de arboles de regresión

In [19]:
from sklearn.tree import DecisionTreeRegressor

initial_time = time.time()

dtr=DecisionTreeRegressor(random_state=454455)
dtr.fit(X_train, y_train)
prediction = dtr.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data.append([])
data[-1].append("Arbol de regresión")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 515.0023430301002 R2: 0.4034051451640329 Time: 0.20699858665466309


In [20]:
initial_time = time.time()
param_grid = {"criterion":["squared_error", "absolute_error"], "max_depth":list(range(3,7,2)), "min_samples_split":list(range(41,50,2)), "min_samples_leaf":list(range(1,6,2))}

grid_tree = GridSearchCV(DecisionTreeRegressor(random_state=454455), param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_tree.fit(X_train, y_train)
prediction = grid_tree.best_estimator_.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_tree.best_params_)

RMSE: 450.4297858772788 R2: 0.5436318816916665 Time: 62.60784029960632 Params: {'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 45}


### Evaluación de regresión lineal

In [21]:
from sklearn.linear_model import LinearRegression

initial_time = time.time()
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('lr', LinearRegression())]
)
pipe.fit(X_train, y_train)
prediction = pipe.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data.append([])
data[-1].append("Regresión lineal")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 551.5411322396272 R2: 0.3157466907413353 Time: 0.037999629974365234


In [22]:
initial_time = time.time()
param_grid = {"lr__fit_intercept":[True, False], "lr__positive": [True, False]}
grid_lr = GridSearchCV(pipe, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_lr.fit(X_train, y_train)
prediction = grid_lr.best_estimator_.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_lr.best_params_)

RMSE: 551.5411322396272 R2: 0.3157466907413353 Time: 0.31764698028564453 Params: {'lr__fit_intercept': True, 'lr__positive': False}


### Evaluación de regresión lineal Lasso

In [23]:
from sklearn.linear_model import Lasso, LassoCV

initial_time = time.time()
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('lasso', Lasso(max_iter=10000, random_state=454455))]
)

pipe.fit(X_train, y_train)
prediction = pipe.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data.append([])
data[-1].append("Regresión Lineal Lasso")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 556.312686615394 R2: 0.3038561000354465 Time: 0.1555957794189453


In [24]:
initial_time = time.time()
#print(list(np.logspace(-1, 0.5, 30)))

param_grid = {"lasso__alpha": list(np.logspace(-1, 0.5, 30)), "lasso__fit_intercept": [True, False]}
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('lasso', Lasso(max_iter=15000, random_state=454455))]
)
grid_lasso = GridSearchCV(pipe, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_lasso.fit(X_train, np.ravel(y_train))
prediction = grid_lasso.best_estimator_.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_lasso.best_params_)

RMSE: 556.8195678819071 R2: 0.3025869466817208 Time: 53.11083221435547 Params: {'lasso__alpha': 1.0826367338740546, 'lasso__fit_intercept': True}


### Evaluación de SVM

In [25]:
from sklearn.svm import SVR

initial_time = time.time()
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('svm', SVR())]
)

pipe.fit(X_train, np.ravel(y_train))
prediction = pipe.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data.append([])
data[-1].append("SVM")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 637.5113391363395 R2: 0.08580904225322772 Time: 2.0426087379455566


In [26]:
initial_time = time.time()
param_grid = {"svm__kernel":["linear", "poly", "rbf"],"svm__C": list(range(790,810,5))}
pipe = Pipeline([
    ('scaler', RobustScaler()), 
    ('svm', SVR())]
)
grid_svm = GridSearchCV(pipe, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_svm.fit(X_train, np.ravel(y_train))
prediction = grid_svm.best_estimator_.predict(X_test)
rmse = root_mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_svm.best_params_)

RMSE: 422.9510859225257 R2: 0.5976153709246705 Time: 496.5190794467926 Params: {'svm__C': 805, 'svm__kernel': 'rbf'}


### Resumen de resultados y conclusiones

In [27]:
from tabulate import tabulate
headers = ["Name", "Default params RMSE", "Default params r2", "Time", "RMSE con HPO", "r2 con HPO", "Time"]
for i in range(len(data)):
    for j in range(len(data[i])):
        if j >0:
            data[i][j] = round(data[i][j], 3)
    
print(tabulate(data, headers=headers, tablefmt="pretty"))

+------------------------+---------------------+-------------------+-------+--------------+------------+---------+
|          Name          | Default params RMSE | Default params r2 | Time  | RMSE con HPO | r2 con HPO |  Time   |
+------------------------+---------------------+-------------------+-------+--------------+------------+---------+
|          KNN           |       434.51        |       0.575       | 0.126 |   414.242    |   0.614    |  3.289  |
|   Arbol de regresión   |       515.002       |       0.403       | 0.207 |    450.43    |   0.544    | 62.608  |
|    Regresión lineal    |       551.541       |       0.316       | 0.038 |   551.541    |   0.316    |  0.318  |
| Regresión Lineal Lasso |       556.313       |       0.304       | 0.156 |    556.82    |   0.303    | 53.111  |
|          SVM           |       637.511       |       0.086       | 2.043 |   422.951    |   0.598    | 496.519 |
+------------------------+---------------------+-------------------+-------+----